In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
import xgboost
import pickle as pkl
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


In [2]:
train = pd.read_csv(r'C:\Users\Dell\panic_disorder_dataset_training.csv') 
train.head()

,Participant ID,Age,Gender,Family History,Personal History,Current Stressors,Symptoms,Severity,Impact on Life,Demographics,Medical History,Psychiatric History,Substance Use,Coping Mechanisms,Social Support,Lifestyle Factors,Panic Disorder Diagnosis
0,1,38,Male,No,Yes,Moderate,Shortness of breath,Mild,Mild,Rural,Diabetes,Bipolar disorder,None,Socializing,High,Sleep quality,0
1,2,51,Male,No,No,High,Panic attacks,Mild,Mild,Urban,Asthma,Anxiety disorder,Drugs,Exercise,High,Sleep quality,0
2,3,32,Female,Yes,No,High,Panic attacks,Mild,Significant,Urban,Diabetes,Depressive disorder,None,Seeking therapy,Moderate,Exercise,0
3,4,64,Female,No,No,Moderate,Chest pain,Moderate,Moderate,Rural,Diabetes,None,None,Meditation,High,Exercise,0
4,5,31,Male,Yes,No,Moderate,Panic attacks,Mild,Moderate,Rural,Asthma,None,Drugs,Seeking therapy,Low,Sleep quality,0


In [3]:
test = pd.read_csv(r'C:\Users\Dell\panic_disorder_dataset_testing.csv') 
test.head()

,Participant ID,Age,Gender,Family History,Personal History,Current Stressors,Symptoms,Severity,Impact on Life,Demographics,Medical History,Psychiatric History,Substance Use,Coping Mechanisms,Social Support,Lifestyle Factors,Panic Disorder Diagnosis
0,1,41,Male,Yes,No,High,Shortness of breath,Mild,Mild,Urban,Diabetes,Bipolar disorder,Alcohol,Seeking therapy,Low,Exercise,0
1,2,20,Female,Yes,No,Low,Shortness of breath,Mild,Significant,Urban,Asthma,Anxiety disorder,Drugs,Exercise,High,Diet,0
2,3,32,Male,Yes,Yes,High,Panic attacks,Severe,Mild,Rural,Heart disease,Bipolar disorder,Drugs,Meditation,Moderate,Exercise,0
3,4,41,Female,Yes,Yes,Moderate,Shortness of breath,Moderate,Significant,Urban,Heart disease,Anxiety disorder,None,Exercise,High,Sleep quality,0
4,5,36,Female,Yes,No,High,Chest pain,Severe,Significant,Rural,Asthma,Depressive disorder,None,Seeking therapy,Low,Exercise,0


In [4]:
print('Train data shape:', train.shape) 
print('Test data shape:', test.shape)

Train data shape: (100000, 17)
Test data shape: (20000, 17)


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   Participant ID            100000 non-null  int64 
 1   Age                       100000 non-null  int64 
 2   Gender                    100000 non-null  object
 3   Family History            100000 non-null  object
 4   Personal History          100000 non-null  object
 5   Current Stressors         100000 non-null  object
 6   Symptoms                  100000 non-null  object
 7   Severity                  100000 non-null  object
 8   Impact on Life            100000 non-null  object
 9   Demographics              100000 non-null  object
 10  Medical History           100000 non-null  object
 11  Psychiatric History       100000 non-null  object
 12  Substance Use             100000 non-null  object
 13  Coping Mechanisms         100000 non-null  object
 14  Socia

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Participant ID            20000 non-null  int64 
 1   Age                       20000 non-null  int64 
 2   Gender                    20000 non-null  object
 3   Family History            20000 non-null  object
 4   Personal History          20000 non-null  object
 5   Current Stressors         20000 non-null  object
 6   Symptoms                  20000 non-null  object
 7   Severity                  20000 non-null  object
 8   Impact on Life            20000 non-null  object
 9   Demographics              20000 non-null  object
 10  Medical History           20000 non-null  object
 11  Psychiatric History       20000 non-null  object
 12  Substance Use             20000 non-null  object
 13  Coping Mechanisms         20000 non-null  object
 14  Social Support        

In [7]:
train.isnull().sum()

Participant ID              0
Age                         0
Gender                      0
Family History              0
Personal History            0
Current Stressors           0
Symptoms                    0
Severity                    0
Impact on Life              0
Demographics                0
Medical History             0
Psychiatric History         0
Substance Use               0
Coping Mechanisms           0
Social Support              0
Lifestyle Factors           0
Panic Disorder Diagnosis    0
dtype: int64

In [8]:
test.isnull().sum()

Participant ID              0
Age                         0
Gender                      0
Family History              0
Personal History            0
Current Stressors           0
Symptoms                    0
Severity                    0
Impact on Life              0
Demographics                0
Medical History             0
Psychiatric History         0
Substance Use               0
Coping Mechanisms           0
Social Support              0
Lifestyle Factors           0
Panic Disorder Diagnosis    0
dtype: int64

In [9]:
# categorical columns
cat_cols = []
for i in train.columns:
    if train[i].dtype=='object': 
        cat_cols.append(i)
cat_cols

['Gender',
 'Family History',
 'Personal History',
 'Current Stressors',
 'Symptoms',
 'Severity',
 'Impact on Life',
 'Demographics',
 'Medical History',
 'Psychiatric History',
 'Substance Use',
 'Coping Mechanisms',
 'Social Support',
 'Lifestyle Factors']

In [10]:
le = {}
for column in train.columns:
    if train [column].dtype == object:
        le[column]={}
        c=0
        for i in train [column].unique():
            le[column][i]=c
            c+=1
        train[column]=train [column].map(le[column])
le = {}
for column in test.columns:
    if test[column].dtype == object:
        le[column]={}
        c=0
        for i in test[column].unique():
            le[column][i]=c
            c+=1
        test[column]=test[column].map(le[column])

In [29]:
import pandas as pd
import numpy as np
from collections import Counter

# Assuming 'Panic Disorder Diagnosis' is a Series, not a DataFrame
# Convert it to a DataFrame
y_train = pd.DataFrame({'Panic Disorder Diagnosis': y_train})

# Replace 'Panic Disorder Diagnosis' with the actual name of your target variable
target_variable = 'Panic Disorder Diagnosis'

# Check if the target variable is present in y_train
if target_variable not in y_train.columns:
    raise KeyError(f"The target variable '{target_variable}' does not exist in y_train.")

# Use the correct column name in the rest of your code
indices = np.random.choice(x_train[y_train[target_variable] == 0].index, size=x_train[y_train[target_variable] == 1].shape[0], replace=False)
temp_x = x_train[y_train[target_variable] == 0].loc[indices]
temp_x = temp_x.append(x_train[y_train[target_variable] == 1])
temp_y = y_train[y_train[target_variable] == 0].loc[indices]
temp_y = temp_y.append(y_train[y_train[target_variable] == 1])

print(x_train.shape, y_train.shape)
print("Before balancing", Counter(y_train[target_variable]))
print(temp_x.shape, temp_y.shape)
print("After balancing", Counter(temp_y[target_variable]))


(100000, 16) (100000, 1)
Before balancing Counter({0: 95715, 1: 4285})
(8570, 16) (8570, 1)
After balancing Counter({0: 4285, 1: 4285})


In [30]:
f_p_values=chi2(temp_x, temp_y)
f_p_values

(array([1.02720201e+05, 1.34247188e+00, 1.44634994e+00, 9.58064581e+01,
        1.67684344e+02, 1.85571793e-03, 9.77453626e+01, 2.89790785e+02,
        4.59640996e-01, 3.30330910e+01, 1.15365031e+01, 4.39563810e+01,
        2.95795212e+01, 7.52895782e+00, 3.40016799e+01, 4.46300000e+03]),
 array([0.00000000e+00, 2.46598518e-01, 2.29114539e-01, 1.26681791e-22,
        2.37109357e-38, 9.65639321e-01, 4.75796619e-23, 5.52258507e-65,
        4.97791802e-01, 9.06035954e-09, 6.82428805e-04, 3.35775893e-11,
        5.36689807e-08, 6.07150204e-03, 5.50645100e-09, 0.00000000e+00]))

In [31]:
#p Values
p_values=pd.Series(f_p_values[1])
p_values.index=x_train.columns
p_values.sort_index(ascending=True)

Age                    2.465985e-01
Coping Mechanisms      6.071502e-03
Current Stressors      9.656393e-01
Demographics           9.060360e-09
Family History         1.266818e-22
Gender                 2.291145e-01
Impact on Life         4.977918e-01
Lifestyle Factors      0.000000e+00
Medical History        6.824288e-04
Participant ID         0.000000e+00
Personal History       2.371094e-38
Psychiatric History    3.357759e-11
Severity               5.522585e-65
Social Support         5.506451e-09
Substance Use          5.366898e-08
Symptoms               4.757966e-23
dtype: float64

In [32]:
fts = ["Coping Mechanisms", "Current Stressors", "Demographics", "Family History", "Gender", "Impact on Life", "Symptoms"]
temp_x[fts]

,Coping Mechanisms,Current Stressors,Demographics,Family History,Gender,Impact on Life,Symptoms
59753,0,1,1,1,0,0,2
67416,2,2,0,1,1,2,3
20447,1,1,1,0,1,2,0
59457,0,2,1,0,1,0,2
41359,2,1,0,1,0,2,0
...,...,...,...,...,...,...,...
99826,0,0,0,1,0,1,1
99965,2,1,1,1,1,1,3
99979,1,2,1,1,0,2,1
99980,3,0,0,1,0,2,1


In [33]:
train.describe(include='all')

,Participant ID,Age,Gender,Family History,Personal History,Current Stressors,Symptoms,Severity,Impact on Life,Demographics,Medical History,Psychiatric History,Substance Use,Coping Mechanisms,Social Support,Lifestyle Factors,Panic Disorder Diagnosis
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.00000,100000.00000
mean,50000.500000,41.454300,0.499480,0.500420,0.502100,1.00187,2.009280,1.001450,0.998810,0.499130,1.500850,1.49780,1.000710,1.498020,0.997410,1.00259,0.04285
std,28867.657797,13.839204,0.500002,0.500002,0.499998,0.81628,1.416529,0.817774,0.817224,0.500002,1.118638,1.11785,0.817433,1.120129,0.816376,0.81488,0.20252
min,1.000000,18.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000
25%,25000.750000,29.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000
50%,50000.500000,41.000000,0.000000,1.000000,1.000000,1.00000,2.000000,1.000000,1.000000,0.000000,2.000000,1.00000,1.000000,1.000000,1.000000,1.00000,0.00000
75%,75000.250000,53.000000,1.000000,1.000000,1.000000,2.00000,3.000000,2.000000,2.000000,1.000000,2.000000,2.00000,2.000000,3.000000,2.000000,2.00000,0.00000
max,100000.000000,65.000000,1.000000,1.000000,1.000000,2.00000,4.000000,2.000000,2.000000,1.000000,3.000000,3.00000,2.000000,3.000000,2.000000,2.00000,1.00000


In [39]:
def train_models_eval(temp_x,temp_y,fts):
    print("\n--->RANDOM FOREST")
    rf = RandomForestClassifier(random_state=1234)
    rf.fit(temp_x[fts],temp_y)
    y_pred=rf.predict(x_test[fts])
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print("SCORE:",rf.score(x_test[fts],y_test))
    
    print("\n--->DECISION TREE")
    dtf = DecisionTreeClassifier(random_state=1234)
    dtf.fit(temp_x[fts],temp_y)
    y_pred-dtf.predict(x_test[fts])
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print("SCORE:",dtf.score(x_test[fts],y_test))
    
    print("\n--->KNN")
    knn = KNeighborsClassifier() 
    knn.fit(temp_x[fts],temp_y)
    y_pred-knn.predict(x_test[fts])
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred)) 
    print("SCORE:",knn.score(x_test[fts],y_test))
    
    print("\n--->EXTRAS TREES CLASSIFIER")
    etc= ExtraTreesClassifier(random_state=1234)
    etc.fit(temp_x[fts],temp_y)
    y_pred=etc.predict(x_test[fts])
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred)) 
    print("SCORE:",etc.score(x_test[fts],y_test))
    
    print("\n--->XGBOOST")
    xgb = xgboost.XGBClassifier()
    xgb.fit(temp_x[fts],temp_y) y_pred=xgb.predict(x_test[fts])
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print("SCORE:",xgb.score(x_test[fts],y_test))
    return rf,dtf,knn,etc,xgb

In [40]:
rf,dtf, knn, etc,xgb = train_models_eval(temp_x,temp_y,fts)


--->RANDOM FOREST
[[13086  6073]
 [  564   277]]
              precision    recall  f1-score   support

           0       0.96      0.68      0.80     19159
           1       0.04      0.33      0.08       841

    accuracy                           0.67     20000
   macro avg       0.50      0.51      0.44     20000
weighted avg       0.92      0.67      0.77     20000

SCORE: 0.66815

--->DECISION TREE
[[13086  6073]
 [  564   277]]
              precision    recall  f1-score   support

           0       0.96      0.68      0.80     19159
           1       0.04      0.33      0.08       841

    accuracy                           0.67     20000
   macro avg       0.50      0.51      0.44     20000
weighted avg       0.92      0.67      0.77     20000

SCORE: 0.68785

--->KNN
[[13086  6073]
 [  564   277]]
              precision    recall  f1-score   support

           0       0.96      0.68      0.80     19159
           1       0.04      0.33      0.08       841

    accuracy

In [43]:
knn.predict([temp_x[fts].iloc[60,:]])

array([1])

In [50]:
y_test['Panic Disorder Diagnosis'][60]

KeyError: 'Panic Disorder Diagnosis'

In [51]:
dtf.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 1234,
 'splitter': 'best'}

In [57]:
param_grid = {
'criterion': ['gini', 'entropy'],
'max_depth': [None, 5, 10, 15],
'min_samples_split': [2, 5, 10],
'min_samples_leaf': [1, 2, 3],
'max_features': [None, 'sqrt', 'log2']
}
# Create a Decision Tree classifier
dt_classifier =DecisionTreeClassifier(random_state=1234)
# Create GridSearchCV object
grid_search = GridSearchCV(dt_classifier, param_grid, cv=5, verbose = 1, n_jobs=-1)
# Fit the data to perform grid search
grid_search.fit(temp_x[fts], temp_y)
# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score",grid_search.best_score_)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Hyperparameters: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Score 0.7929988331388564


In [59]:
y_pred=grid_search.best_estimator_.predict(x_test[fts])
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("SCORE:",grid_search.best_estimator_.score(x_test[fts],y_test))

[[13496  5663]
 [  580   261]]
              precision    recall  f1-score   support

           0       0.96      0.70      0.81     19159
           1       0.04      0.31      0.08       841

    accuracy                           0.69     20000
   macro avg       0.50      0.51      0.44     20000
weighted avg       0.92      0.69      0.78     20000

SCORE: 0.68785


In [80]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 1234,
 'verbose': 0,
 'warm_start': False}

In [65]:
param_grid = {
'max_depth': [None, 5, 10],
'n_estimators': [50, 100, 2001],
'min_samples_leaf': [1, 2, 4], 
'max_features': ['sqrt', 'log2'],
'min_samples_split': [2, 5, 10],
}
#Create the RandomForestClassifier
rf_classifier =RandomForestClassifier(random_state=1234)
# Create the GridSearchCV object
GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, verbose=1, n_jobs=4)
grid_search
# Fit the GridSearchCV to your data
grid_search.fit(temp_x[fts], temp_y)
# Print the best parameters and best score
print("Best Parameters: ", grid_search.best_params_) 
print("Best Score: ", grid_search.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Parameters:  {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Score:  0.7929988331388564


In [66]:
y_pred-grid_search.best_estimator_.predict(x_test[fts])
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred)) 
print("SCORE:",grid_search.best_estimator_.score(x_test[fts],y_test))

[[13496  5663]
 [  580   261]]
              precision    recall  f1-score   support

           0       0.96      0.70      0.81     19159
           1       0.04      0.31      0.08       841

    accuracy                           0.69     20000
   macro avg       0.50      0.51      0.44     20000
weighted avg       0.92      0.69      0.78     20000

SCORE: 0.68785


In [67]:
xgb.get_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [77]:
import xgboost
from sklearn.model_selection import GridSearchCV

params = {
    'min_child_weight': [10, 20],
    'gamma': [0, 1.5, 2.0],
    'colsample_bytree': [0.6, 0.8, 0.9],
    'max_depth': [4, 5, 6]
}

xgb = xgboost.XGBClassifier(learning_rate=0.5, n_estimators=100, objective='binary:logistic', nthread=3)

fitmodel = GridSearchCV(xgb, param_grid=params, cv=5, refit=True, scoring="accuracy", n_jobs=-1, verbose=3)
fitmodel.fit(temp_x[fts], temp_y)

# Print the best parameters and best score
print("Best Parameters: ", fitmodel.best_params_)
print("Best Score: ", fitmodel.best_score_)


Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Parameters:  {'colsample_bytree': 0.8, 'gamma': 1.5, 'max_depth': 6, 'min_child_weight': 20}
Best Score:  0.8054842473745625


In [78]:
y_pred-fitmodel.best_estimator_.predict(x_test[fts])
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("SCORE:",fitmodel.best_estimator_.score(x_test[fts],y_test))

[[13496  5663]
 [  580   261]]
              precision    recall  f1-score   support

           0       0.96      0.70      0.81     19159
           1       0.04      0.31      0.08       841

    accuracy                           0.69     20000
   macro avg       0.50      0.51      0.44     20000
weighted avg       0.92      0.69      0.78     20000

SCORE: 0.6434


In [79]:
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [86]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [20, 30, 40],
    'p': [1, 2]
}

# Create the KNeighborsClassifier
knn = KNeighborsClassifier()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, verbose=1, n_jobs=4)

# Fit the GridSearchCV to your data
grid_search.fit(temp_x[fts], temp_y)

# Print the best parameters and best score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)


Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best Parameters:  {'algorithm': 'brute', 'leaf_size': 20, 'n_neighbors': 7, 'p': 2, 'weights': 'distance'}
Best Score:  0.7889148191365226


In [89]:
y_pred=grid_search.best_estimator_.predict(x_test[fts])
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("SCORE:",grid_search.best_estimator_.score(x_test[fts],y_test))

[[13365  5794]
 [  580   261]]
              precision    recall  f1-score   support

           0       0.96      0.70      0.81     19159
           1       0.04      0.31      0.08       841

    accuracy                           0.68     20000
   macro avg       0.50      0.50      0.44     20000
weighted avg       0.92      0.68      0.78     20000

SCORE: 0.6813


In [95]:
knn =KNeighborsClassifier(algorithm= 'brute', leaf_size= 20, n_neighbors=6, p= 1, weights ='distance')
knn.fit(temp_x[fts],temp_y)
y_pred-knn.predict(x_test[fts])
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("SCORE:",knn.score(x_test[fts],y_test))

[[13365  5794]
 [  580   261]]
              precision    recall  f1-score   support

           0       0.96      0.70      0.81     19159
           1       0.04      0.31      0.08       841

    accuracy                           0.68     20000
   macro avg       0.50      0.50      0.44     20000
weighted avg       0.92      0.68      0.78     20000

SCORE: 0.70705


In [97]:
pkl.dump(knn,open('knn.pkl','wb'))